In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import os

In [2]:
rel_dir = '/home/soyorh/Desktop/studyforrest/Preprocessing steps/'

In [3]:
# os.chdir(rel_dir)
# !ls

## # Functions

In [4]:
def get_signals_from_run(sub_id, set_t, roi, run_i, signal_selection):
    signals_run_i = np.transpose(np.load(f'{rel_dir}{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
    signals_run_i = signals_run_i[signal_selection,  :]

    return signals_run_i

In [5]:
# QS: astype('float32') vs astype(np.float32)

In [6]:
def save_stacked_signals(sub_id, set_t, roi, time_range=5, shift=4 ):
    
    '''
    
    time_range: how many frames you want as input in the time dimension.
    shift: how many delays in frames.
    
    '''
    
    number_of_run_files = len(glob(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run*'))
    signals_list = []

    for run_i in tqdm(range(1, number_of_run_files+1)):
      
        # leftOut_frames = shift + time_range  ('Cause fmri duration == movie duration) already done in target space
        seen_img =  np.load(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run{run_i}.npy') #[:-leftOut_frames]
        n_frames = seen_img.shape[0]
        signal_selection = np.arange(time_range)[np.newaxis, :] + np.arange(shift, shift+n_frames)[:, np.newaxis]
        
        signals_run_i = get_signals_from_run(sub_id, set_t, roi, run_i, signal_selection)
        signals_list.append(signals_run_i)
        
        print(signals_run_i.shape)

    signals = np.concatenate(signals_list).astype('float32')
    signals = signals[:, :, :, np.newaxis, np.newaxis].astype(np.float32)
    signals.shape   

    os.makedirs('shiftedStuck_fMRI', exist_ok=True)
    os.makedirs(f'shiftedStuck_fMRI/sub-{sub_id}/{set_t[:-3]}', exist_ok=True)

    np.save(f'{rel_dir}shiftedStuck_fMRI/sub-{sub_id}/{set_t[:-3]}/{roi}-shift={shift},time_range={time_range}', signals)

## # Saving

In [7]:
save_stacked_signals('18', 'testing', 'VIS', time_range=5, shift=4)

 38%|███▊      | 3/8 [00:00<00:00, 10.59it/s]

(442, 5, 216)
(432, 5, 216)
(429, 5, 216)


 62%|██████▎   | 5/8 [00:00<00:00, 11.57it/s]

(479, 5, 216)
(453, 5, 216)
(430, 5, 216)


100%|██████████| 8/8 [00:00<00:00, 10.98it/s]

(534, 5, 216)
(328, 5, 216)


In [8]:
save_stacked_signals('18', 'training', 'VIS', time_range=5, shift=4)

 38%|███▊      | 3/8 [00:00<00:00,  7.93it/s]

(442, 5, 216)
(432, 5, 216)
(429, 5, 216)


 62%|██████▎   | 5/8 [00:00<00:00,  9.57it/s]

(479, 5, 216)
(453, 5, 216)
(430, 5, 216)


100%|██████████| 8/8 [00:00<00:00,  9.97it/s]

(534, 5, 216)
(328, 5, 216)


### # check the data

In [9]:
sub_id = '18'
set_t = 'testing'
np.load(f'shiftedStuck_fMRI/sub-{sub_id}/{set_t[:4]}/VIS-shift=4,time_range=5.npy').shape

(3527, 5, 216, 1, 1)